# Overview

This notebook mimics the Genesis battery charging profile for the internal battery. 

## Battery Characteristics

The battery specification does not contain a discharge profile, which is unusual.The battery characteristics critical to charging are:

* Li-Ion Chemistry
* 500 mAh Capacity (4.2 V → 3.0 V, 0.2 C normalized discharge current)
* 4.2 V Charge Voltage

## Genesis Charging Characteristics

The charger implements the following charge profile:

* Constant Current Mode
    * 247 mA
    * V<sub>BAT</sub>3.0V - 4.1V
* Constant Voltage Mode
    * V<sub>BAT</sub>=4.1 V
    * I<sub>TERM</sub>=24.7 mA

Note that the charging voltage is reduced from the specification in an effort to improve the battery life.

## RS-232 Configuration

I am using the Keithley 2401 for my test automation target. Its RS232 port is configured as follows:

* baud: 9600 
* bits: 8-bits
* parity: none
* terminator: CR/LF

## References

* [procedures](https://pymeasure.readthedocs.io/en/latest/api/instruments/keithley/keithley2400.html)

In [1]:
# All measurements will be taken using the pymeasure library.
import pymeasure
pymeasure.__version__

'0.9.0'

In [2]:
from pymeasure.instruments.keithley import Keithley2400 #The Keithley 2401 communicates using the Keithley 2400 library
import pyvisa as visa            #provides RS232 support
import numpy as np               #provides array support. I may not need this library.
import pandas as pd              #provides data analysis tools
import matplotlib.pyplot as plt  #python graphics package
from time import sleep           #provides support for generating delays
sourcemeter = Keithley2400("ASRL3::INSTR")  #using RS 232 COM3

# Constant Current Charge Test

In [3]:
# Constant Current Charge Test

from time import sleep, strftime, time
from datetime import datetime

# Setup up the CV Termination Voltage
vTerm = 4.15

# Setup Meter
sourcemeter.reset()                        # Clear the interface
sourcemeter.use_front_terminals()          # Use the front-terminals
sourcemeter.apply_current()                # Sets up to source current
sourcemeter.source_current_range = 300e-3  # Sets the source current range to 10 mA
sourcemeter.compliance_voltage = 10        # Sets the compliance voltage to 10 V
sourcemeter.source_current = 247e-3        # Sets the source current to 0 mA
sourcemeter.enable_source()                # Enables the source output
sourcemeter.measure_voltage()              # Sets up to measure voltage

# Grab Data, Put in File and Dataframe
df = pd.DataFrame({'time': [],
                    'voltage': []})
print("START")
vBAT = sourcemeter.voltage
with open("CC_Charge_Bat_Voltage.csv", "a") as log:
    log.write("{0},{1}\n".format("time","voltage"))
    while vBAT<=vTerm:
        now=datetime.now()
        vBAT = sourcemeter.voltage
        df = df.append(pd.DataFrame({'time':[now], 'voltage':[vBAT]}),ignore_index=True)
        log.write("{0},{1}\n".format(now.strftime("%Y-%m-%d %H:%M:%S"),str(vBAT)))
        print("{0},{1}".format(now.strftime("%Y-%m-%d %H:%M:%S"),str(vBAT)))
        sleep(30)
print("END")
sourcemeter.disable_source()                # Disable the source output             
df.to_csv('CC_Charge_.csv', index=False)

START
2022-02-16 10:10:56,3.462523
2022-02-16 10:11:26,3.620406
2022-02-16 10:11:56,3.651968
2022-02-16 10:12:27,3.669858
2022-02-16 10:12:57,3.68089
2022-02-16 10:13:27,3.688228
2022-02-16 10:13:57,3.693692
2022-02-16 10:14:27,3.69813
2022-02-16 10:14:57,3.702094
2022-02-16 10:15:27,3.705806
2022-02-16 10:15:58,3.70944
2022-02-16 10:16:28,3.71301
2022-02-16 10:16:58,3.716573
2022-02-16 10:17:28,3.720114
2022-02-16 10:17:58,3.723631
2022-02-16 10:18:28,3.727144
2022-02-16 10:18:58,3.730725
2022-02-16 10:19:29,3.734227
2022-02-16 10:19:59,3.737708
2022-02-16 10:20:29,3.741141
2022-02-16 10:20:59,3.744521
2022-02-16 10:21:29,3.74784
2022-02-16 10:21:59,3.751046
2022-02-16 10:22:29,3.754168
2022-02-16 10:22:59,3.757225
2022-02-16 10:23:30,3.760209
2022-02-16 10:24:00,3.763081
2022-02-16 10:24:30,3.765882
2022-02-16 10:25:00,3.768559
2022-02-16 10:25:30,3.771132
2022-02-16 10:26:00,3.773595
2022-02-16 10:26:30,3.775985
2022-02-16 10:27:00,3.778261
2022-02-16 10:27:31,3.780507
2022-02-16 10

# Constant Voltage Portion

During the constant voltage portion of the charge cycle, the battery is driven from a constant voltage source (4.1 V) with the charging terminated when I<sub>BAT</sub> is 24.7 mA.

In [4]:
from time import sleep, strftime, time
from datetime import datetime

# Setup Meter
sourcemeter.reset()                        # Reset the interface
sourcemeter.use_front_terminals()          # Work from the front terminals
sourcemeter.apply_voltage()                # We are going to apply voltage and measure current
sourcemeter.source_voltage_range = 4.2     # Sets the source current range to 10 mA
sourcemeter.compliance_current= 1.0        # Sets the compliance voltage to 10 V
sourcemeter.source_voltage = vTerm         # The constant voltage is vTerm
sourcemeter.enable_source()                # Enables the source output (apply voltage)
sourcemeter.measure_current()              # Sets up to measure voltage (measure the current)

# Grab Data, Put in File and Dataframe

dfCV = pd.DataFrame({'time': [],
                     'current': []})
print("START")
iBAT = sourcemeter.current
with open("CV_Bat_Current.csv", "a") as log:
    log.write("{0},{1}\n".format("time","current"))
    while iBAT>=24.7e-3:
        now=datetime.now()
        iBAT = sourcemeter.current
        dfCV = dfCV.append(pd.DataFrame({'time':[now], 'current':[iBAT]}),ignore_index=True)
        log.write("{0},{1}\n".format(now.strftime("%Y-%m-%d %H:%M:%S"),str(iBAT)))
        print("{0},{1}".format(now.strftime("%Y-%m-%d %H:%M:%S"),str(iBAT)))
        sleep(30)
print("END")
sourcemeter.disable_source()                # Disable the source output             
dfCV.to_csv('CV_Portion.csv', index=False)

START
2022-02-16 11:42:08,0.4258394
2022-02-16 11:42:38,0.3360486
2022-02-16 11:43:08,0.3018934
2022-02-16 11:43:39,0.2780859
2022-02-16 11:44:09,0.2600686
2022-02-16 11:44:39,0.245885
2022-02-16 11:45:09,0.234568
2022-02-16 11:45:39,0.2251154
2022-02-16 11:46:09,0.2170573
2022-02-16 11:46:39,0.210088
2022-02-16 11:47:09,0.2038227
2022-02-16 11:47:39,0.1980491
2022-02-16 11:48:09,0.1927154
2022-02-16 11:48:40,0.1877364
2022-02-16 11:49:10,0.182944
2022-02-16 11:49:40,0.1784146
2022-02-16 11:50:10,0.1740488
2022-02-16 11:50:40,0.1699127
2022-02-16 11:51:10,0.1659211
2022-02-16 11:51:40,0.1620353
2022-02-16 11:52:10,0.1582408
2022-02-16 11:52:40,0.154561
2022-02-16 11:53:11,0.151032
2022-02-16 11:53:41,0.1475742
2022-02-16 11:54:11,0.1442305
2022-02-16 11:54:41,0.1409496
2022-02-16 11:55:11,0.1377841
2022-02-16 11:55:41,0.134687
2022-02-16 11:56:11,0.131694
2022-02-16 11:56:41,0.1288012
2022-02-16 11:57:11,0.1259977
2022-02-16 11:57:42,0.1232511
2022-02-16 11:58:12,0.1205969
2022-02-16 1